<a href="https://colab.research.google.com/github/SamuelMbogo/Colab_projects/blob/main/Bond_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# Mapping for payment frequencies to periods per year
FREQ_MAP = {
    'monthly': 12,
    'quarterly': 4,
    'semi-annually': 2,
    'annually': 1
}

def calculate_bond_price(coupon_rate, principal, required_return, tax_rate, maturity, frequency):
    """
    Calculate the present value of a bond with tax-adjusted coupon payments.

    Parameters:
    -----------
    coupon_rate : float
        Annual coupon rate as a decimal (e.g., 0.0625 for 6.25%)
    principal : float
        Bond's face value/principal amount
    required_return : float
        Required rate of return as a decimal
    tax_rate : float
        Tax rate as a decimal (e.g., 0.20 for 20%)
    maturity : int
        Time to maturity in years
    frequency : str
        Frequency of coupon payments ('monthly', 'quarterly', 'semi-annually', 'annually')

    Returns:
    --------
    float
        Present value of the bond rounded to 3 decimal places

    Raises:
    -------
    ValueError
        If frequency is not one of the accepted payment frequencies
    """
    if frequency not in FREQ_MAP:
        raise ValueError(f"Invalid payment frequency. Must be one of: {', '.join(FREQ_MAP.keys())}")

    # Get number of payments per year from frequency map
    payments_per_year = FREQ_MAP[frequency]

    # Calculate adjusted rates and payments
    adj_coupon_rate = coupon_rate * (1 - tax_rate)
    periodic_coupon = (adj_coupon_rate * principal) / payments_per_year
    periodic_rate = required_return / payments_per_year

    # Calculate total number of periods
    total_periods = maturity * payments_per_year

    # Generate time periods array
    periods = np.arange(1, total_periods + 1)

    # Calculate present value of all coupon payments
    coupon_pv = periodic_coupon * np.sum(1 / (1 + periodic_rate) ** periods)

    # Calculate present value of principal
    principal_pv = principal / (1 + periodic_rate) ** total_periods

    # Return total present value rounded to 3 decimal places
    return round(coupon_pv + principal_pv, 3)

def calculate_duration(coupon_rate, principal, required_return, tax_rate, maturity, frequency):
    """
    Calculate the Macaulay duration of a bond.

    Parameters:
    -----------
    Same as calculate_bond_price()

    Returns:
    --------
    float
        Duration of the bond in years, rounded to 3 decimal places

    Notes:
    ------
    Duration measures the weighted average time until cash flows are received,
    using present values as weights. It indicates the bond's price sensitivity
    to yield changes.
    """
    if frequency not in FREQ_MAP:
        raise ValueError(f"Invalid payment frequency. Must be one of: {', '.join(FREQ_MAP.keys())}")

    payments_per_year = FREQ_MAP[frequency]

    # Calculate essential values
    adj_coupon_rate = coupon_rate * (1 - tax_rate)
    periodic_coupon = (adj_coupon_rate * principal) / payments_per_year
    periodic_rate = required_return / payments_per_year
    total_periods = maturity * payments_per_year

    # Create time points array (in years)
    time_points = np.arange(1, total_periods + 1) / payments_per_year

    # Create cash flows array (all coupons, with principal added to final payment)
    cash_flows = np.full(total_periods, periodic_coupon)
    cash_flows[-1] += principal

    # Calculate present value factors
    pv_factors = 1 / (1 + periodic_rate) ** np.arange(1, total_periods + 1)

    # Calculate present values of all cash flows
    present_values = cash_flows * pv_factors

    # Calculate duration
    weighted_time_values = np.sum(time_points * present_values)
    total_pv = np.sum(present_values)

    return round(weighted_time_values / total_pv, 3)

# Example usage with the test values
if __name__ == "__main__":
    # Test parameters
    test_params = {
        'coupon_rate': 0.0625,    # 6.25%
        'principal': 1000,        # $1,000
        'required_return': 0.04,  # 4%
        'tax_rate': 0.20,        # 20%
        'maturity': 15,          # 15 years
        'frequency': 'quarterly'  # Quarterly payments
    }

    # Calculate and display results
    bond_price = calculate_bond_price(**test_params)
    bond_duration = calculate_duration(**test_params)

    print(f'Bond price = {bond_price:.3f}')
    print(f'Duration = {bond_duration:.3f}')

Bond price = 1112.388
Duration = 10.900
